In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Carregar o dataset
df = pd.read_csv("../data/rent.csv")

# Separar features (X) e target (y)
X = df.drop("price", axis=1)  # ou o nome correto da coluna de preço
y = df["price"]

print("a")

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)


# Criar pipelines para cada modelo
# Modelo 1: Linear Regression com scaler
pipeline1 = Pipeline([("scaler", StandardScaler()), ("model", LinearRegression())])

# Modelo 2: MLP sem ativação com scaler
pipeline2 = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "model",
            MLPRegressor(
                hidden_layer_sizes=(5, 5),
                activation="identity",
                max_iter=1000,
                random_state=42,
            ),
        ),
    ]
)

# Modelo 3: MLP com ReLU e scaler
pipeline3 = Pipeline(
    [
        ("scaler", StandardScaler()),
        (
            "model",
            MLPRegressor(
                hidden_layer_sizes=(5, 5),
                activation="relu",
                max_iter=1000,
                random_state=42,
            ),
        ),
    ]
)

print("b")

# Aplicar cross-validation aos pipelines
cv_results1 = cross_validate(
    pipeline1, X, y, cv=5, scoring=mae_scorer, return_train_score=True
)

print("1")

cv_results2 = cross_validate(
    pipeline2, X, y, cv=5, scoring=mae_scorer, return_train_score=True
)

print("2")

cv_results3 = cross_validate(
    pipeline3, X, y, cv=5, scoring=mae_scorer, return_train_score=True
)

print("3")

# Extrair os MAE (converter para positivo)
mae1 = -cv_results1["test_score"]
mae2 = -cv_results2["test_score"]
mae3 = -cv_results3["test_score"]

# Preparar os dados para o boxplot
data = [mae1, mae2, mae3]
labels = ["Linear Regression", "MLP (no activation)", "MLP (ReLU)"]

# Criar o boxplot
plt.figure(figsize=(10, 6))
plt.boxplot(data, labels=labels)
plt.ylabel("Test MAE")
plt.title("Comparison of Models - 5-Fold Cross-Validation")
plt.grid(True, alpha=0.3)
plt.show()

a


NameError: name 'mae_scorer' is not defined